In [ ]:
!wget -O ml-100k.zip https://tinyurl.com/y5ynqofz
!unzip ml-100k.zip

In [ ]:
user_item_df = pd.read_csv(user_item, names=['iid', 'uid', 'rating', 'timestamp'], sep='\t')
user_item_df = user_item_df.merge(item_df, on=['iid'])
user_item_df = user_item_df.merge(user_df, on=['uid'])
user_item_df.head() 

In [ ]:
import numpy 
import matplotlib.pyplot as plt
import seaborn as sns

def plot_heat_map(df, figsize=(10,7)): 
    df = df.div(df.sum(axis=1), axis=0)     
    plt.subplots(figsize=figsize)
    sns.heatmap(df)

gender = user_item_df[user_item_df['rating']>3][['gender']+genres].groupby(['gender']).sum()
plot_heat_map(gender, figsize=(10,2))
    

In [ ]:


occupation = user_item_df[user_item_df['rating']>3][['occupation']+genres].groupby(['occupation']).sum()
plot_heat_map(occupation, figsize=(10,7))

In [ ]:
user_item_df['age_segment']=user_item_df['age']//10
age = user_item_df[user_item_df['rating']>3][['age_segment']+genres].groupby(['age_segment']).sum()
plot_heat_map(age, figsize=(10,5))

In [2]:
import sys
sys.path.append('../')

from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.xgboost_transformer import  XGBoostTransformer

user_path = './ml-100k/u.user'
item_path = './ml-100k/u.item'
user_item = './ml-100k/u.data'
reader = IMDBDataReader()
user_item  = reader.read_user_item_rating(user_item)
users = reader.read_user_data(user_path)
items = reader.read_item_data(item_path)
train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]
transformer = XGBoostTransformer(users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(users, items, test_user_item)

In [ ]:
X_train

In [ ]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2

In [3]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/xgboost-movielens'
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')
test_key       = 'test.protobuf'
test_prefix    = '{}/{}/'.format(prefix, 'test')
output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)


import io,boto3
import sagemaker.amazon.common as smac

def writeToLocalCSV(X, key, Y): 
    f_name = key + ".csv"
    output = open(f_name, 'w')


    for i in range(0, len(X)): 
        line = ''
        raw = X[i]
        line += str(int(Y[i])) 
        line += ' '
        for ele in raw:
            line += str(ele[0])+':'+str(ele[1])+' '
        line += '\n'
        output.write(line)    
    return f_name  



def writeDatasetToCSV(X, bucket, prefix, key, d_type, Y=None):
    f_name = writeToLocalCSV(X, key, Y)
    obj = '{}/{}'.format(prefix, f_name)
    boto3.client('s3').upload_file(f_name, bucket, obj)
    return 's3://{}/{}'.format(bucket,obj)
 
print(X_train)    
train_data = writeDatasetToCSV(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
  
print('Output: {}'.format(output_prefix))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output: s3://recommendation-demo-yianc/sagemaker/xgboost-movielens/output


In [ ]:
train_data 

In [4]:
region = 'us-east-1'

In [5]:
import sagemaker 

container = sagemaker.image_uris.retrieve(framework='xgboost', region=region, version='latest')
container 


'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'

In [6]:
%%time
import boto3
from time import gmtime, strftime

job_name = 'DEMO-xgboost-regression-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)
crole = 'arn:aws:iam::230755935769:role/service-role/AmazonSageMaker-ExecutionRole-20200603T105247'
#Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": crole,
    "OutputDataConfig": {
        "S3OutputPath": 's3://'+bucket + "/" + prefix + "/single-xgboost"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m5.2xlarge",
        "VolumeSizeInGB": 5
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "max_depth":"3",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
        "objective":"binary:logistic",
        "num_round":"5000"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://'+bucket+'/'+train_prefix,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        }
    ]
}


client = boto3.client('sagemaker', region_name='us-east-1') # to be modified
client.create_training_job(**create_training_params)

import time

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)
while status !='Completed' and status!='Failed':
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print(status)

Training job DEMO-xgboost-regression-2020-08-14-05-59-28
InProgress
InProgress
InProgress
InProgress
Completed
CPU times: user 165 ms, sys: 52.1 ms, total: 217 ms
Wall time: 4min 7s


In [7]:
%%time
import boto3
from time import gmtime, strftime

model_name=job_name + '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = crole,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])




DEMO-xgboost-regression-2020-08-14-05-59-28-model
s3://recommendation-demo-yianc/sagemaker/xgboost-movielens/single-xgboost/DEMO-xgboost-regression-2020-08-14-05-59-28/output/model.tar.gz
arn:aws:sagemaker:us-east-1:230755935769:model/demo-xgboost-regression-2020-08-14-05-59-28-model
CPU times: user 10.9 ms, sys: 4.59 ms, total: 15.5 ms
Wall time: 1.09 s


In [8]:
runtime_client = boto3.client('runtime.sagemaker', region_name=region)

In [9]:
from time import gmtime, strftime

endpoint_config_name = 'DEMO-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])



DEMO-XGBoostEndpointConfig-2020-08-14-06-03-37
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:230755935769:endpoint-config/demo-xgboostendpointconfig-2020-08-14-06-03-37


In [10]:
%%time
import time

endpoint_name = 'DEMO-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
while status=='Creating':
    print("Status: " + status)
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)


DEMO-XGBoostEndpoint-2020-08-14-06-03-37
arn:aws:sagemaker:us-east-1:230755935769:endpoint/demo-xgboostendpoint-2020-08-14-06-03-37
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:230755935769:endpoint/demo-xgboostendpoint-2020-08-14-06-03-37
Status: InService
CPU times: user 280 ms, sys: 279 ms, total: 559 ms
Wall time: 27min 17s


In [11]:
%%time
import json
from itertools import islice
import math
import struct


f_name = writeToLocalCSV(X_test, 'test', Y_test)

f = open(f_name, 'r')
with open(f_name, 'r') as f:    
    for l in f.readlines(): 
        payload = l 
        print(payload)
        response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/x-libsvm', 
                                   Body=payload)
        result = response['Body'].read()
        result = result.decode("utf-8")
        result = result.split(',')
        result = [math.ceil(float(i)) for i in result]
        label = payload.strip(' ').split()[0]
        print ('Label: ',label,'\nPrediction: ', result[0])

0 0:23 1:1 5:1 24:1997 26:1 41:1 44:59 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1998 26:1 41:1 44:6 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1997 38:1 41:1 44:57 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1997 33:1 44:10 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1997 40:1 41:1 44:11 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1997 30:1 44:16 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1996 33:1 41:1 44:29 

Label:  0 
Prediction:  1
0 0:23 1:1 5:1 24:1998 36:1 44:2 

Label:  0 
Prediction:  1
1 0:23 1:1 5:1 24:1997 27:1 41:1 44:35 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1997 26:1 38:1 39:1 41:1 44:110 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1998 31:1 33:1 41:1 44:7 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1997 31:1 33:1 38:1 44:64 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1996 30:1 44:45 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1997 26:1 38:1 40:1 41:1 44:22 

Label:  1 
Prediction:  1
1 0:23 1:1 5:1 24:1998 31:1 33:1 41:1 44:14 

Label:  

Label:  1 
Prediction:  1
1 0:24 2:1 4:1 24:1993 30:1 39:1 44:108 

Label:  1 
Prediction:  1
1 0:24 2:1 4:1 24:1994 26:1 27:1 40:1 44:36 

Label:  1 
Prediction:  1
0 0:24 2:1 4:1 24:1995 30:1 44:6 

Label:  0 
Prediction:  1
1 0:24 2:1 4:1 24:1995 30:1 39:1 44:83 

Label:  1 
Prediction:  1
1 0:24 2:1 4:1 24:1991 26:1 27:1 40:1 44:64 

Label:  1 
Prediction:  1
0 0:24 2:1 4:1 24:1993 26:1 27:1 30:1 44:26 

Label:  0 
Prediction:  1
1 0:24 2:1 4:1 24:1994 30:1 33:1 44:15 

Label:  1 
Prediction:  1
0 0:24 2:1 4:1 24:1990 29:1 30:1 44:38 

Label:  0 
Prediction:  1
0 0:24 2:1 4:1 24:1996 26:1 27:1 34:1 44:45 

Label:  0 
Prediction:  1
0 0:24 2:1 4:1 24:1993 30:1 44:12 

Label:  0 
Prediction:  1
0 0:24 2:1 4:1 24:1990 26:1 39:1 44:10 

Label:  0 
Prediction:  1
0 0:24 2:1 4:1 24:1997 38:1 41:1 44:52 

Label:  0 
Prediction:  1
1 0:24 2:1 4:1 24:1971 27:1 29:1 37:1 44:15 

Label:  1 
Prediction:  1
0 0:24 2:1 4:1 24:1995 30:1 44:34 

Label:  0 
Prediction:  1
0 0:24 2:1 4:1 24:1988 28:

Label:  1 
Prediction:  1
0 0:27 2:1 4:1 24:1985 30:1 40:1 44:199 

Label:  0 
Prediction:  1
1 0:27 2:1 4:1 24:1996 33:1 41:1 44:135 

Label:  1 
Prediction:  1
1 0:27 2:1 4:1 24:1939 33:1 39:1 42:1 44:92 

Label:  1 
Prediction:  1
1 0:27 2:1 4:1 24:1989 26:1 27:1 40:1 41:1 44:70 

Label:  1 
Prediction:  1
0 0:27 2:1 4:1 24:1995 33:1 44:34 

Label:  0 
Prediction:  1
0 0:27 2:1 4:1 24:1993 33:1 41:1 44:55 

Label:  0 
Prediction:  1
1 0:27 2:1 4:1 24:1994 26:1 30:1 43:1 44:51 

Label:  1 
Prediction:  1
0 0:27 2:1 4:1 24:1995 26:1 27:1 29:1 34:1 40:1 44:31 

Label:  0 
Prediction:  1
1 0:27 2:1 4:1 24:1951 26:1 27:1 39:1 42:1 44:95 

Label:  1 
Prediction:  1
1 0:27 2:1 4:1 24:1993 30:1 39:1 44:108 

Label:  1 
Prediction:  1
0 0:27 2:1 4:1 24:1995 29:1 33:1 44:15 

Label:  0 
Prediction:  1
0 0:27 2:1 4:1 24:1991 26:1 27:1 40:1 44:64 

Label:  0 
Prediction:  1
0 0:27 2:1 4:1 24:1996 33:1 39:1 44:111 

Label:  0 
Prediction:  1
0 0:27 2:1 4:1 24:1941 28:1 29:1 37:1 44:47 

Label:  

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1993 33:1 44:83 

Label:  1 
Prediction:  1
0 0:29 1:1 23:1 24:1993 33:1 39:1 44:24 

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1981 26:1 33:1 38:1 39:1 41:1 44:27 

Label:  1 
Prediction:  1
0 0:29 1:1 23:1 24:1993 33:1 39:1 44:78 

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1994 30:1 39:1 44:17 

Label:  1 
Prediction:  1
0 0:29 1:1 23:1 24:1994 30:1 39:1 44:45 

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1992 36:1 44:3 

Label:  1 
Prediction:  1
0 0:29 1:1 23:1 24:1990 33:1 44:7 

Label:  0 
Prediction:  1
0 0:29 1:1 23:1 24:1991 30:1 42:1 44:14 

Label:  0 
Prediction:  1
0 0:29 1:1 23:1 24:1976 33:1 44:1 

Label:  0 
Prediction:  1
0 0:29 1:1 23:1 24:1992 26:1 39:1 42:1 44:54 

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1993 30:1 39:1 44:40 

Label:  1 
Prediction:  1
0 0:29 1:1 23:1 24:1995 33:1 39:1 44:32 

Label:  0 
Prediction:  1
1 0:29 1:1 23:1 24:1989 30:1 39:1 44:166 

Label:  1 
Prediction:  1
1 0:29 1:1 23:1 24:1995 

Label:  0 
Prediction:  1
1 0:27 1:1 8:1 24:1997 30:1 33:1 44:21 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1998 33:1 41:1 44:61 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1998 31:1 33:1 41:1 44:7 

Label:  1 
Prediction:  1
0 0:27 1:1 8:1 24:1996 33:1 41:1 44:29 

Label:  0 
Prediction:  1
1 0:27 1:1 8:1 24:1991 33:1 44:14 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1993 33:1 44:4 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1965 37:1 44:107 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1996 33:1 44:22 

Label:  1 
Prediction:  1
1 0:27 1:1 8:1 24:1980 26:1 27:1 33:1 39:1 40:1 42:1 44:245 

Label:  1 
Prediction:  1
0 0:27 1:1 8:1 24:1982 26:1 27:1 40:1 44:128 

Label:  0 
Prediction:  1
0 0:27 1:1 8:1 24:1982 33:1 37:1 42:1 44:48 

Label:  0 
Prediction:  1
0 0:27 1:1 8:1 24:1993 26:1 27:1 40:1 44:136 

Label:  0 
Prediction:  1
0 0:27 1:1 8:1 24:1989 33:1 44:106 

Label:  0 
Prediction:  1
1 0:27 1:1 8:1 24:1985 30:1 44:36 

Label:  1 
Prediction:  1
0 0:27 1:1 8:1 24:1994 

...

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1993 26:1 27:1 31:1 44:23 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 33:1 44:16 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1997 33:1 40:1 44:269 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 33:1 40:1 44:220 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1993 30:1 44:12 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1993 26:1 41:1 44:219 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 40:1 44:7 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 30:1 44:68 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 26:1 33:1 42:1 44:188 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1993 33:1 42:1 44:207 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1997 30:1 39:1 44:71 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1996 33:1 44:157 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1994 30:1 31:1 34:1 44:41 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1995 26:1 27:1 44:11 

Label:  1 
Prediction:  1
1 0:24 1:1 8:1 24:1994 26:1 39:1 41:1 44:

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1996 30:1 44:7 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1994 26:1 30:1 44:3 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1996 30:1 31:1 44:10 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1993 30:1 40:1 44:2 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1996 30:1 44:14 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1995 30:1 44:10 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1994 29:1 30:1 44:1 

Label:  0 
Prediction:  1
0 0:30 1:1 9:1 24:1995 30:1 36:1 44:3 

Label:  0 
Prediction:  1
1 0:35 1:1 3:1 24:1970 33:1 42:1 44:72 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1979 26:1 36:1 40:1 41:1 44:180 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1994 30:1 39:1 42:1 44:191 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1954 38:1 41:1 44:135 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1994 26:1 27:1 30:1 39:1 44:95 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1988 26:1 41:1 44:139 

Label:  1 
Prediction:  1
1 0:35 1:1 3:1 24:1986 26:1

Label:  0 
Prediction:  1
1 0:38 1:1 13:1 24:1993 30:1 39:1 44:154 

Label:  1 
Prediction:  1
0 0:38 1:1 13:1 24:1993 30:1 39:1 44:108 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1994 30:1 39:1 44:10 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1988 30:1 44:138 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1995 30:1 33:1 39:1 44:31 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1995 30:1 39:1 44:73 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1990 30:1 33:1 44:45 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1994 30:1 39:1 44:28 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1994 26:1 30:1 43:1 44:51 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1994 30:1 44:47 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1985 30:1 44:52 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1979 30:1 44:66 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1974 30:1 44:41 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1970 30:1 42:1 44:122 

Label:  0 
Prediction:  1
0 0:38 1:1 13:1 24:1993 30:1 44:7

Label:  0 
Prediction:  1
1 0:51 1:1 7:1 24:1997 26:1 33:1 39:1 44:171 

Label:  1 
Prediction:  1
1 0:51 1:1 7:1 24:1997 33:1 44:13 

Label:  1 
Prediction:  1
0 0:51 1:1 7:1 24:1997 33:1 39:1 44:5 

Label:  0 
Prediction:  1
0 0:20 2:1 8:1 24:1995 27:1 29:1 44:14 

Label:  0 
Prediction:  1
0 0:20 2:1 8:1 24:1994 30:1 44:16 

Label:  0 
Prediction:  1
0 0:20 2:1 8:1 24:1995 33:1 41:1 44:1 

Label:  0 
Prediction:  1
1 0:20 2:1 8:1 24:1957 39:1 44:17 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1995 28:1 29:1 30:1 44:266 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1944 38:1 41:1 44:14 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1995 30:1 33:1 39:1 44:81 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1993 33:1 44:73 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1995 33:1 39:1 44:171 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1995 29:1 30:1 33:1 44:128 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1954 38:1 41:1 44:135 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1995 33:1 44:31

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 33:1 42:1 44:112 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1997 30:1 39:1 44:16 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 33:1 39:1 44:111 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1997 30:1 44:93 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 36:1 41:1 44:194 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1992 26:1 39:1 42:1 44:54 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 33:1 39:1 44:91 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1972 26:1 31:1 33:1 44:282 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 30:1 44:46 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1980 26:1 27:1 33:1 39:1 40:1 42:1 44:245 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1990 26:1 41:1 44:122 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1996 33:1 44:115 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1958 38:1 41:1 44:129 

Label:  1 
Prediction:  1
1 0:20 2:1 8:1 24:1981 26:1 27:1 44:284 

Label:  1 
Prediction:  1
1 0:20 2:1 8

In [ ]:
# X_test, Y_test,X_cold_test, Y_cold_test
from sagemaker_utils.query_serializer import SparseFormatSerializer 
# from sklearn.metrics import mean_squared_error
import numpy 

sparse = SparseFormatSerializer(nFeatures)
fm_predictor.serializer = sparse
fm_predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

def model_rmse(X_test, Y_test): 
    X_test_arr = X_test
        
    result = fm_predictor.predict(X_test_arr) 
    y_pred = [] 
    for p in result['predictions']: 
        y_pred.append(p['score'])
    
    return numpy.sqrt(numpy.mean((y_pred-Y_test)**2))

from sklearn.metrics import accuracy_score

def model_accuracy(X_test, Y_test): 
    X_test_arr = X_test
        
    result = fm_predictor.predict(X_test_arr) 
    print(result)
    y_pred = [] 
    for p in result['predictions']: 
        if p['score'] > 0.5:
            y_pred.append(1)
        else: 
            y_pred.append(0)
    return accuracy_score(Y_test, y_pred, normalize=False)


In [ ]:
rmse = model_accuracy(X_train[:3000], Y_train[:3000]) / len(Y_train[:3000])
rmse

In [ ]:
rmse = model_accuracy(X_cold_test, Y_cold_test) / len(Y_cold_test)
rmse

In [ ]:
genres = item_df[['iid']+genres].set_index('iid')

In [ ]:
popularity = user_item_df[user_item_df['rating']>3].groupby('iid').count()['uid'].to_dict() 
answer = {} 
for ui in test_user_item: 
    uid = ui[0]
    iid = ui[1]
    rating = ui[2]
    if uid not in answer:  
        answer[uid] = set() 
    if rating > 3: 
        answer[uid].add(iid)
    
all_pop = 0 
hits = 0 
topk = 10
valid_user_nb = 0 
rcmded = set()
for uid in answer.keys():
    if uid in transformer.u_idx: 
        cuidx = transformer.u_idx[uid] 
        u_one_json = json.loads(results[cuidx])
        pred = set()
        valid_user_nb += 1 
        for i, movieid in enumerate(u_one_json['labels'][:topk]): 
            rcmded.add(movieid)
            if int(item_inv_idx[movieid]) in popularity: 
                all_pop += popularity[int(item_inv_idx[movieid])] 
            pred.add(item_inv_idx[movieid])
        hits += len(pred.intersection(answer[uid]))
print(all_pop/topk/valid_user_nb)
print(len(answer))
print(hits)
print(len(rcmded))